In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.scanner import ScannerSubscription
from ib_insync import *
import time
import threading

from collections import deque

## Stock Scanner
- NOTE: this file should only be ran during market hours - there is no way to run this using historical data
- File selects top 5 stocks (this number can be changed later, currently hardcoded) based off of top percent gain metric (also can be changed to a different metric if needed)
- File then shows user the current top 5 stocks, with a refresh of the top percent gain metric happening in 30 second intervals
- User will be shown only the top 5 stocks at a time, and can select "n" if they are not satisfied with the picked stocks. Program will automatically run with updated stocks every 30 seconds and will terminate when user types in "y", indicating they like the stocks displayed and want to trade those stocks
- File will then locally save the user's selected stocks

In [2]:
class TradeApp(EWrapper, EClient):
    
    def __init__(self):
        EClient.__init__(self,self)
        self.stockStack = deque()
    
    def scannerData(self, reqId, rank, contractDetails, distance, benchmark, projection, legsStr):
        super().scannerData(reqId, rank, contractDetails, distance, benchmark, projection, legsStr)
        getSymbol = contractDetails.contract.symbol
        self.stockStack.append(getSymbol)

In [3]:
def usStkScan(asset_type="STK",asset_loc="STK.US.MAJOR",scan_code = "TOP_PERC_GAIN"):
    scanSub = ScannerSubscription()
    scanSub.numberOfRows = 5
    scanSub.abovePrice = 10
    scanSub.belowPrice = 200
    scanSub.aboveVolume = 1000000
    scanSub.instrument = asset_type
    scanSub.locationType = asset_loc
    scanSub.scanCode = scan_code
    return scanSub

In [4]:
def websocket_con():
    app.run()

In [5]:
app = TradeApp()
app.connect(host = '127.0.0.1', port = 7497, clientId = 23)
con_thread = threading.Thread(target=websocket_con)
con_thread.start()
time.sleep(1)

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [7]:
app.reqScannerSubscription(1,usStkScan(),[],[])
time.sleep(30)

In [ ]:
traderLike = False
pickedStocks = None

while not traderLike:
    
    app.reqScannerSubscription(1,usStkScan(),[],[]) #append to stack regardless
    
    viewStocks = app.stockStack #view current stack
    print(viewStocks)
    currStocks = []
    
    for i in range(5):
        ret = viewStocks.pop()
        currStocks.append(ret)
    
    time.sleep(30)
    print("5 stocks: ",currStocks)
    val = input("Do you want to trade these 5 stocks? y/n")
    
    if val == "y":
        traderLike = True
        pickedStocks = currStocks
        app.disconnect()
        print("Congrats! Your picked stocks were: ",pickedStocks)
        print("Happy Trading :)")
        break
    
    currStocks = [] #reinitialize list to hold the top of stack 
    

ERROR 1 322 Error processing request.-'cv' : cause - Duplicate ticker ID for API scanner subscription


deque(['NUVL', 'RUM', 'DXCM', 'FBC', 'TECL'])
5 stocks:  ['TECL', 'FBC', 'DXCM', 'RUM', 'NUVL']
Do you wanna trade these 5 stocks? y/nn


ERROR 1 322 Error processing request.-'cv' : cause - Duplicate ticker ID for API scanner subscription


deque(['NUVL', 'RUM', 'DXCM', 'FBC', 'TECL', 'NUVL', 'RUM', 'DXCM', 'FBC', 'TECL'])
5 stocks:  ['TECL', 'FBC', 'DXCM', 'RUM', 'NUVL']


In [ ]:
app.disconnect()

In [ ]:
pickedStocks 